<a href="https://colab.research.google.com/github/SalimBobo/SalimBoBoMohirDev/blob/main/SalimBoBo_Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/sample_data

In [ ]:
# Import necessary libraries
import torch
import fastai
from fastai.vision.all import *
from ipywidgets import widgets
import os
from PIL import Image
import numpy as np
import pandas as pd
from google.colab import drive
from zipfile import ZipFile

print(torch.__version__)
print(fastai.__version__)

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths to the datasets
train_zip_path = '/content/drive/MyDrive/train.zip'
test_zip_path = '/content/drive/MyDrive/test.zip'

# Extract the datasets
with ZipFile(train_zip_path, 'r') as zip:
  zip.extractall('/content/train')
  print("Train dataset is extracted")

with ZipFile(test_zip_path, 'r') as zip:
  zip.extractall('/content/test')
  print("Test dataset is extracted")

# Define the path
path = Path('/content/train')

# Create a DataBlock with data augmentation
db = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),  # Increase image resolution
    batch_tfms=aug_transforms(size=224)  # Apply data augmentation
)

dls = db.dataloaders(path)

# Train the model
learn = vision_learner(dls, resnet101, metrics=accuracy)  # Use a more powerful model
learn.fine_tune(6)  # Fine-tune for more epochs

# Unfreeze and train more
learn.unfreeze()
learn.fit_one_cycle(4, lr_max=slice(1e-6,1e-4))

# Interpret the model
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

# Predict on the test set
path = Path('/content/test')
images = [os.path.join(path, f) for f in os.listdir(path)]

predictions = []
for i in images:
  pred, pred_id, _ = learn.predict(i)
  predictions.append(pred)

# Export the predictions
df1 = pd.DataFrame(images)
df2 = pd.DataFrame(predictions)
df = pd.concat([df1, df2], axis=1)

df.to_csv('/content/submission_modified.csv')
